In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from datetime import datetime
import calendar

In [ ]:
#Extract data from FHV, HV-FHV,and Open Meteo for month of July
filename = "https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2023-07.parquet"

response = requests.get(filename)

if response.status_code == 200:
    print("File exists")
else:
    print("File not found")


File exists


In [ ]:
df = pd.read_parquet(filename)
print(df.head())

  dispatching_base_num     pickup_datetime    dropOff_datetime  PUlocationID  \
0               B00112 2023-07-01 00:08:46 2023-07-01 00:15:08           NaN   
1               B00112 2023-07-01 00:37:54 2023-07-01 19:45:55           NaN   
2               B00112 2023-07-01 00:03:13 2023-07-01 00:04:54           NaN   
3               B00112 2023-07-01 00:57:45 2023-07-01 01:01:17           NaN   
4               B00149 2023-07-01 00:03:17 2023-07-01 00:14:50           NaN   

   DOlocationID  SR_Flag Affiliated_base_number  
0          14.0      NaN                 B00112  
1         132.0      NaN                 B00112  
2          14.0      NaN                 B00112  
3          14.0      NaN                 B00112  
4         181.0      NaN                 B00149  


In [ ]:
print(df.tail())

        dispatching_base_num     pickup_datetime    dropOff_datetime  \
1370838               B03388 2023-07-31 23:34:25 2023-08-01 00:14:11   
1370839               B03391 2023-07-31 23:59:00 2023-08-01 00:07:50   
1370840               B03408 2023-07-31 23:37:43 2023-07-31 23:59:36   
1370841               B03482 2023-07-31 23:00:38 2023-07-31 23:06:01   
1370842               B03482 2023-07-31 23:16:27 2023-07-31 23:32:03   

         PUlocationID  DOlocationID  SR_Flag Affiliated_base_number  
1370838           NaN         100.0      NaN                 B03388  
1370839           NaN         160.0      NaN                 B00647  
1370840           NaN          41.0      NaN                 B03408  
1370841           NaN          37.0      NaN                 B03482  
1370842           NaN          65.0      NaN                 B03482  


In [ ]:
df.shape

(1370843, 7)

In [ ]:
print(df.columns)

Index(['dispatching_base_num', 'pickup_datetime', 'dropOff_datetime',
       'PUlocationID', 'DOlocationID', 'SR_Flag', 'Affiliated_base_number'],
      dtype='object')


In [ ]:
print(df.dtypes)

dispatching_base_num              object
pickup_datetime           datetime64[ns]
dropOff_datetime          datetime64[ns]
PUlocationID                     float64
DOlocationID                     float64
SR_Flag                          float64
Affiliated_base_number            object
dtype: object


In [ ]:
#What is the significance of SR_Flag and PUloactionID?
missingNum = 0
hasNum = 0

for x in df['SR_Flag']:
  if pd.isna(x):
    missingNum += 1
  else:
    hasNum +=1

print(f'SR_Flag Count\nAmount of NaNs: {missingNum} \t Amount of rows populated: {hasNum}')

SR_Flag Count
Amount of NaNs: 1370843 	 Amount of rows populated: 0


In [ ]:
missingNum = 0
hasNum = 0

for x in df['PUlocationID']:
  if pd.isna(x):
    missingNum += 1
  else:
    hasNum +=1

print(f'PUlocationID Count\nAmount of NaNs: {missingNum} \t Amount of rows populated: {hasNum}')

PUlocationID Count
Amount of NaNs: 1053778 	 Amount of rows populated: 317065


In [ ]:
count = 0

for x in df['dispatching_base_num']:
  if not df['dispatching_base_num'].equals(df['Affiliated_base_number']):
    count += 1

print(f'Total count where the dipatch and affiliated base numbers did not match:\n{count}')

affCount = count

Total count where the dipatch and affiliated base numbers did not match:
1370843


In [ ]:
def DOW ():

  given_date = datetime.strptime(str(df['pickup_datetime'][1370842]), '%Y-%m-%d %H:%M:%S')
  day_of_week = given_date.weekday()
  if day_of_week == 0:
    day = "Monday"
  elif day_of_week == 1:
    day = "Tuesday"
  elif day_of_week == 2:
    day = "Wednesday"
  elif day_of_week == 3:
    day = "Thursday"
  elif day_of_week == 4:
    day = "Friday"
  elif day_of_week == 5:
    day = "Saturday"
  elif day_of_week == 6:
    day = "Sunday"

  return print(day)

#July 1st was a Saturday
#...Monday

DOW()

Monday


In [ ]:
#Aggregate total trips based on a date
%%time

def total_trip_Day():
  print("***" * 24)
  print("**** DISCLAIMER: Range of data from July -> December 2023 permitted ****")
  print("***" * 24)
  month = int(input('\nEnter month by number, e.g. July = 7:\t'))
  day = int(input('\nEnter day by number as well:\t'))

  #search through df and match every trip with the month and day, keep count
  #for every item in my df pick up time col where the month and day match
  count = 0
  for i in range(len(df)):
    if df['pickup_datetime'][i] >= datetime(2023, month, day, 0, 0, 0) and df['pickup_datetime'][i] <= datetime(2023, month, day, 23, 59, 59):
      count += 1
  print("---" * 15)
  print(f"\nTotal trips on {month}/{day}/2023: {count:,} trips\n")
  print("---" * 15)


total_trip_Day()

************************************************************************
**** DISCLAIMER: Range of data from July -> December 2023 permitted ****
************************************************************************

Enter month by number, e.g. July = 7:	7

Enter day by number as well:	1
---------------------------------------------

Total trips on 7/1/2023: 39,703 trips

---------------------------------------------
CPU times: user 37 s, sys: 98.9 ms, total: 37.1 s
Wall time: 44.3 s


In [ ]:
#Does my total count of July match up with my total count of indices
count = 0
for i in range(len(df)):
  if df['pickup_datetime'][i] >= datetime(2023, 7, 1, 0, 0, 0) and df['pickup_datetime'][i] <= datetime(2023, 7, 31, 23, 59, 59):
    count += 1
print(f'{count:,}')

1,370,843


In [ ]:
#Data validated
for i in range(len(df)):
  pass
i += 1
print(f'{i:,}')

1,370,843


-Filter by Borough


-Total trips by taxi type


-Total trips by taxi type by month, by day


-Total trips by boroughs


-Revenue by Boroughs


In [ ]:
#return trips by destination borough
def boroughFilter():
  locID = []

  for x in df['DOlocationID']:
    if x != 'nan':
      locID.append(x)

  print(locID)

boroughFilter()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

